In [ ]:
from ETL.Functions.api_yelp import *
import pandas as pd

ruta_bucket = "gs://bucket-steakhouses2"
nombre_archivo = "review.parquet"

business_ids = ['4s4KAGJgtpZn9cgtdWcOmw', 'FN1sLS5hfMydZX94SYD1QA', 'uamAUnL4ox5uXvjquY9wLg']

reviews = []
for business_id in business_ids:
    reviews += API_reviews(business_id)

    df_reviews_api = pd.DataFrame(reviews)
    df_reviews_existentes = reviews_existentes()


    df_reviews_nuevas  = df_reviews_api.loc[~df_reviews_api['review_id'].isin(df_reviews_existentes['review_id'])]

can_reviews = df_reviews_nuevas.shape[0]
print (df_reviews_nuevas.shape[0], 'nuevas reviews')
if can_reviews>0:
    archivo = f'{ruta_bucket}/filtrado/{nombre_archivo}'
    df_reviews_nuevas.to_parquet(archivo, index=False)


In [ ]:
df_reviews_nuevas['date']

In [ ]:
df_reviews_nuevas.info()


In [ ]:
import google.auth
from google.cloud import bigquery

# Create a credentials object
credentials, _ = google.auth.default()

# Create a BigQuery client
client = bigquery.Client(credentials=credentials)



# Crear tablas en BigQuery a partir de archivos
tabla = 'review'
job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET,)

table_id = f'steakhouses2.yelp.{tabla}'
uri = f"{ruta_bucket}/filtrado/{tabla}.parquet"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # API request.

load_job.result()  # Espera que se complete el trabajo.
destination_table = client.get_table(table_id)

In [ ]:

import google.auth
from google.cloud import bigquery

# Create a credentials object
credentials, _ = google.auth.default()

# Create a BigQuery client
client = bigquery.Client(credentials=credentials)




# Insert data from DataFrame into BigQuery
table_ref = client.get_table('steakhouses2.yelp.review')
job = client.load_table_from_dataframe(df_reviews_nuevas, table_ref)
job.result()  # Wait for the job to complete




In [ ]:
import sys

from google.cloud import storage


def check_file_exists(bucket_name, file_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_name)

    exists = blob.exists()
    return exists

ruta_bucket = "gs://bucket-steakhouses2"

print (check_file_exists(ruta_bucket, '/crudo/review.parquet'))






In [ ]:
from google.cloud import storage

def existe_archivo(bucket_name, archivo):
    # Inicializa el cliente de GCS
    client = storage.Client()

    # Obtiene una referencia al bucket
    bucket = client.get_bucket(bucket_name)


    # Verifica si el archivo existe en la carpeta del bucket
    blob = bucket.blob(archivo)
    return blob.exists()

bucket_name = "bucket-steakhouses2"
archivo = 'crudo/yelp/review.parquet'

print (existe_archivo(bucket_name, archivo))

In [ ]:
from ETL.Functions.api_yelp import *
business_a_consultar(10)

In [ ]:
ultimo = '-H8ryT3wGAoHMa9NeZ4tNQ'

import google.auth
from google.cloud import bigquery

# Create a credentials object
credentials, _ = google.auth.default()

# Create a BigQuery client
client = bigquery.Client(credentials=credentials)


table_ref = client.get_table('steakhouses2.yelp.review')



# Crear registro tabla auditoria
#ultimo = df_reviews_nuevas.iloc[-1]['business_id']
audit = client.get_table('steakhouses2.yelp.audit')
row = bigquery.Row([datetime.now(), 'review', "Carga incremental API", 0, table_ref.num_rows, None, 'business_id', ultimo], [])
client.insert_rows(audit, [row])


In [6]:
# Importamos las librerías a usar
import pandas as pd
import google.auth
from google.cloud import bigquery
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize
nltk.download('vader_lexicon') # Descarga vader_lexicon para el análisis de sentimientos.
nltk.download('punkt') # Descarga punkt, un modelo de toquenización que divide el texto en palabras individuales

def sentimientos_Yelp (yelp_as):
    # Convertir las estrellas en score
    def to_sentiment(rating):
        
        rating = int(rating)
        
        # Convert to class
        if rating <= 2:
            return 0
        elif rating == 3:
            return 1
        else:
            return 2

    # Apply to the dataset 
    yelp_as['sentiment'] = yelp_as.stars.apply(to_sentiment)

    # Función para clasificar los textos
    def classify_text(text):
        if 'food' in text.lower():
            return 'comida'
        elif 'service' in text.lower():
            return 'servicio'
        elif 'ambience' in text.lower():
            return 'ambiente'
        return 'otro'

    # Aplica la función a la columna 'text' y crea una nueva columna 'temas'
    yelp_as['temas'] = yelp_as['text'].apply(classify_text)



    analizador = SentimentIntensityAnalyzer()
    def Puntaje_Sentimiento(text):
        tokens = word_tokenize(text)  # Tokenizar el texto
        scores = analizador.polarity_scores(text)  # Obtener los puntajes de sentimiento
        return scores['compound']  # Retornar el puntaje compuesto

    yelp_as['score_sentimientos'] = yelp_as['text'].apply(Puntaje_Sentimiento)

    yelp_as['categorizacion'] = 0  # Inicializamos con 0 por defecto

    # Aplicamos las condiciones para actualizar los valores en la nueva columna
    yelp_as.loc[(yelp_as['score_sentimientos'] > -1) & (yelp_as['score_sentimientos'] < 0), 'categorizacion'] = 0
    yelp_as.loc[yelp_as['score_sentimientos'] == 0, 'categorizacion'] = 1
    yelp_as.loc[(yelp_as['score_sentimientos'] >= 0.1) & (yelp_as['score_sentimientos'] <= 1), 'categorizacion'] = 2


    # Supongamos que tienes un DataFrame llamado 'yelp_as'
    yelp_as['nuevos_scores'] = yelp_as['score_sentimientos'] + 1

    return yelp_as

# Llamar a la función
ruta_bucket = "gs://bucket-steakhouses2"
df = pd.read_parquet(f'{ruta_bucket}/filtrado/yelp/review.parquet')
df = sentimientos_Yelp(df)

# Exporta el dataframe a un archivo Parquet
df.to_parquet(f'{ruta_bucket}/filtrado/yelp/Yelp_Analisis_Sentimiento.parquet', index=False)



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ozzy/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ozzy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
